In [4]:
from foods_and_recipes import foods_df, FOODS_BY_NAME
from nutrition_models import Recipe


target_daily_kj = 7000
meals_per_day = 2
meal_target_kj = target_daily_kj / meals_per_day

protein_name = "Chicken breast, cooked (meat only)"
carb_name = "White rice, cooked (long-grain, enriched)"
veg_name = "Mixed vegetables, frozen (prepared/cooked)"

protein = FOODS_BY_NAME[protein_name]
carb = FOODS_BY_NAME[carb_name]
veg = FOODS_BY_NAME[veg_name]

energy_split = {"protein": 0.45, "carb": 0.35, "veg": 0.2}

def grams_for_energy(food, energy_kj):
    kj_per_gram = food.kj_per_kg / 1000.0
    return energy_kj / kj_per_gram

protein_grams = grams_for_energy(protein, meal_target_kj * energy_split["protein"])
carb_grams = grams_for_energy(carb, meal_target_kj * energy_split["carb"])
veg_grams = grams_for_energy(veg, meal_target_kj * energy_split["veg"])

recipe = Recipe(name=f"{protein.name} + {carb.name} + {veg.name}")
recipe.add(protein, protein_grams)
recipe.add(carb, carb_grams)
recipe.add(veg, veg_grams)

df_foods = foods_df()
df_foods


,name,category,kj_per_kg,dollars_per_kg,notes
0,"Chicken breast, cooked (meat only)",PROTEIN,6903.6,17.61,Energy: roasted chicken breast (USDA via MyFoo...
1,"Chicken thigh, cooked (boneless/skinless)",PROTEIN,7490.0,20.92,Energy: cooked boneless/skinless thigh (USDA v...
2,"White rice, cooked (long-grain, enriched)",CARB,5439.2,0.75,Energy: cooked long-grain white rice (USDA via...
3,"Mixed vegetables, frozen (prepared/cooked)",VEG,3012.5,3.25,Energy: mixed veg frozen unprepared (USDA-styl...
4,"Green beans, cooked",VEG,1757.3,4.00,Energy: cooked green beans (USDA via MyFoodDat...
5,"Carrots, cooked (boiled, drained)",VEG,1475.1,2.60,Energy: cooked carrots (USDA via MyFoodData; c...
6,"Butternut squash, cooked (baked/roasted)",VEG,1673.6,4.39,Energy: baked butternut squash (USDA via MyFoo...


In [5]:
from datetime import date, timedelta
from nutrition_models import DailyMealPlan, grocery_list_from_plans

start_day = date.today()  # change if you want next week

# Build 7 daily plans, 2 servings/day (Oakley is fed twice/day)
week_plans = []
for i in range(7):
    d = start_day + timedelta(days=i)
    plan = DailyMealPlan(day=d)
    for _ in range(meals_per_day):
        plan.add_recipe(recipe)
    week_plans.append(plan)

totals = grocery_list_from_plans(week_plans)  # {FoodItem: grams}

rows = []
for food, grams in totals.items():
    cost = (grams / 1000.0) * food.dollars_per_kg
    rows.append({
        "food": food.name,
        "category": food.category.name,
        "grams": grams,
        "kg": grams / 1000.0,
        "dollars_per_kg": food.dollars_per_kg,
        "cost": cost,
    })

grocery_df = __import__("pandas").DataFrame(rows).sort_values(["category", "food"]).reset_index(drop=True)
grocery_df


,food,category,grams,kg,dollars_per_kg,cost
0,"White rice, cooked (long-grain, enriched)",CARB,3153.037211,3.153037,0.75,2.364778
1,"Chicken breast, cooked (meat only)",PROTEIN,3193.985747,3.193986,17.61,56.246089
2,"Mixed vegetables, frozen (prepared/cooked)",VEG,3253.112033,3.253112,3.25,10.572614


In [6]:
total_week_cost = float(grocery_df["cost"].sum())
per_meal_weight_g = sum(ing.grams for ing in recipe.ingredients)
per_meal_weight_kg = per_meal_weight_g / 1000.0
per_day_weight_kg = per_meal_weight_kg * meals_per_day
weekly_weight_kg = per_day_weight_kg * 7

daily_energy_kj = float(recipe.total_kj * meals_per_day)
daily_energy_kcal = float(recipe.total_kcal * meals_per_day)

print(f"Recipe: {recipe.name}")
print(f"Per-meal: {recipe.total_kj:.0f} kJ ({recipe.total_kcal:.0f} kcal), cost ${recipe.total_cost:.2f}")
print(f"Per-day ({meals_per_day} meals): {daily_energy_kj:.0f} kJ ({daily_energy_kcal:.0f} kcal)")
print(f"Target: {target_daily_kj:.0f} kJ/day")
print(f"Per-meal total weight: {per_meal_weight_g:.0f} g")
print(f"Per-day total weight: {per_day_weight_kg:.2f} kg")
print(f"Total weekly weight: {weekly_weight_kg:.2f} kg")
print(f"Total weekly cost (7 days): ${total_week_cost:.2f}")


Recipe: Chicken breast, cooked (meat only) + White rice, cooked (long-grain, enriched) + Mixed vegetables, frozen (prepared/cooked)
Per-meal: 3500 kJ (837 kcal), cost $4.94
Per-day (2 meals): 7000 kJ (1673 kcal)
Target: 7000 kJ/day
Per-meal total weight: 686 g
Per-day total weight: 1.37 kg
Total weekly weight: 9.60 kg
Total weekly cost (7 days): $69.18
